In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
%cd "/content/drive/MyDrive/CyberTest"

/content/drive/MyDrive/CyberTest


In [3]:
!pwd

/content/drive/MyDrive/CyberTest


In [4]:
!pip -q install contractions
!pip -q install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 21.0 MB/s eta 0:00:00


In [5]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import os
import re
from tqdm import tqdm
import contractions
from textblob import TextBlob
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
binary_model=load_model("models/ArtificialNeuralNetwork.h5",compile=False)
binary_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
multiclass_model=load_model("models/Convolutional_LongShortTermMemory_model.h5",compile=False)
multiclass_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
binaryclass_labels=['Cyberbullying','Normal']
multiclass_labels=['age','ethnicity','gender','other_cyberbullying','religion']

In [9]:
import pickle
with open(file="models/tokens.pkl",mode="rb") as file:
    tok=pickle.load(file=file)

In [28]:
def clean_text(text):
    # expand contraction for words
    text=contractions.fix(text)
    # remove charectir emojes
    emoticons = [r':\)', r':\(', r':P']
    pattern = '|'.join(emoticons)
    text = re.sub(pattern, '', text)
    # remove mentions (@)
    text = re.sub(r'@\w+', '', text)
    # remove hashtags (#)
    text = re.sub(r'#\w+', '', text)
    # remove URLs (http and https)
    text = re.sub(r'https?://\S+', '', text)
    # remove non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    # remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Perform lemmatization on each word in the sentence
    blob = TextBlob(text)
    lemmatized_words = [word.lemmatize() for word in blob.words]
    # Join the lemmatized words back into a sentence
    lemmatized_sentence = " ".join(lemmatized_words)
    # convert to lowercase
    text = lemmatized_sentence.lower()
    return text

In [32]:
user_input=input("Enter a sentence: ")
#normal
#-> College perks, when kids still in school and you
#-> @MaxBlumenthal @Phan_N Always love to hear people show how smart they are by talking about the "inevitable" after the fact.
#age
#Apparently unable to bully them, he dashed to the mics yesterday with his â€œrevisedâ€ school policy The Maryland school attended by President Trumpâ€™s son, Barron, will not fully reopen in September out of concern over #coronavirus https://nyti.ms/3hvT6Pe @nytimes #Kakistocracy
#All the girls in this pic claim their empaths but bullied you in high school
#gender
#I will no longer support Daniel Tosh . Not because of race , rape , or puke jokes , or because he may be gay .
#yes i don't have to but it is funny same as rape jokes, nazi jokes, holocaust joke, black humor etc, that is slippery slope, maybe not funny for you
#religion
#For example, if a Muslim says "it's not a sin to drink, it's only a sin to get drunk, and I don't bet large amounts of money, just a little bit, so it's not harmful gambling." Me: You're an idiot.
#Ghanta one world family. Arjuna led battle against his own cousins. And people like you are useful idiots for the muslims.
#others
#I can never understand what pleasure these idiots get out of bullying and threatening people on twitter.. :(
#I'm not a man, only know how to bully people :(
#ethnicity
#@iMADE_YouMOAN don't play dumb, I should fuck yo little white ass up you nigger. 3chainz.? Aw.
#Why do black women always have big boobs but their nipples are always disgusting? #niggers #women #nips #niggertits

Enter a sentence: sexyness is your style


In [33]:
cleaned_text=clean_text(user_input)
print(cleaned_text)

sexyness is your style


In [34]:
num_data=tok.texts_to_sequences([cleaned_text])
pad_text=pad_sequences(sequences=num_data,maxlen=100,padding="post",truncating="post")

# Model Prediction

In [35]:
print("Level1 started...")
binary_prediction=binary_model.predict(pad_text)
binary_predicted_label=binaryclass_labels[np.argmax(binary_prediction[0])]
print(f"Level 1 Result: {binary_predicted_label}")
if binary_predicted_label!='Normal':
    print(f'Level 2 started...')
    multiclass_prediction=multiclass_model.predict(pad_text)
    multiclass_predicted_label=multiclass_labels[np.argmax(multiclass_prediction[0])]
    print(f"Level 2 Result: {multiclass_predicted_label}")
else:
    print("Completed.")

Level1 started...
1/1 [==============================] - 0s 67ms/step
Level 1 Result: Cyberbullying
Level 2 started...
1/1 [==============================] - 0s 59ms/step
Level 2 Result: other_cyberbullying
